# Allow system to have an sqlite database
This help to keep the conversation going. The system remembers previous messages and when you log out and log back in  
the system still remember the conversation from where you left off. This is because of the use of an sqlite database


In [4]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

In [2]:
load_dotenv()
# llm = ChatGroq(model="llama3-8b-8192")   # 8b model  - faster model
llm = ChatGroq(model="llama3-70b-8192")  # 70b model  x8 better but slower

## Create the schema and node

In [3]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state:BasicChatState)->BasicChatState:
    return {
        "messages": [llm.invoke(state["messages"])]  #
    }

### Create the graph with Memory chechpoint

In [5]:
graph = StateGraph(BasicChatState)

graph.add_node("chatbot", chatbot)
graph.set_entry_point("chatbot")

graph.add_edge("chatbot", END)

# add memory
memory= MemorySaver()
app = graph.compile(checkpointer=memory)

#tie the memory with a thread ID. A unique id that shows the conversation history as it builds
config = {"configurable":{
    "thread_id": 1
}}

### Get the Human question and ensure you tie it to a memory thread

In [7]:
while True:
    user_input = input("User: ")

    if user_input in ["exit","end"]:
        break 
    # remember to pass the configuration, thread id to the invoke method
    result = app.invoke({
        "messages":[HumanMessage(content=user_input)]
    }, config=config)
    # print(result)
    print("AI:",result["messages"][-1].content)


User:  Hi, I am emmanuel


AI: Hi again Emmanuel! I'm happy to help you with your math problems. So, to recap, we did:

1. Added 1 + 5 = 6
2. Multiplied 6 by 4 = 24

Let me know what's the next math problem you'd like to solve!


User:  multiply it by 10


AI: The answer is:

24 × 10 = 240


User:  who is doing this maths


AI: It's me, the AI, and you, Emmanuel! We're doing the math together!


User:  no, I am the one doing the maths


AI: I apologize for the mistake! You're absolutely right, Emmanuel! You're the one doing the math, and I'm just assisting you with the calculations. Great job!


User:  exit
